## Import Libraries

In [3]:
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import os,sys, shutil
import time
from datetime import date
try:
    import cPickle as pickle
except:
    import pickle
import pprint
from collections import deque
from shutil import copyfile
import random
import glob
# Import the required modules
import cv2, os
import numpy as np
from PIL import Image

In [4]:
pkl_fl = open("linkedin_profiles.pickle","rb")
my_original_list=pickle.load(pkl_fl) # errors out here
pkl_fl.close()

In [5]:
my_original_list[0]

{'Full_Name': 'Eryn Olson',
 'Gender': 'Female',
 'Profile_Url': 'https://media.licdn.com/mpr/mpr/shrinknp_400_400/AAEAAQAAAAAAAAbyAAAAJDMzZjQ4NjViLTIwZTctNDM0Yi05ODI5LTk0NmYzZWZmZDI5Yg.jpg',
 'Recommended_Ids': ['erynolson',
  'eryn-olson-62639510a',
  'eryn-olson-432679b1',
  'eryn-olson-1351aa74',
  'courtney-tillman-8b62a664',
  'chance-cozby-13235224',
  'bill-gates-b1a606b0',
  'sarah-eves-2937a719',
  'andrew-solheim-810a3517',
  'andrea-cundiff-76020535',
  'mollie-harper-6b850947',
  'tyler-shaw-934b7',
  'donna-conroy-66987340',
  'paul-wood-73455920'],
 'User_ID': 'eryn-olson-50328143',
 'age': 20}

## label the Images and store into respective folders

In [9]:
directory2 = "Label_Images_Age"

if os.path.exists(directory2):
    shutil.rmtree(directory2)
    os.makedirs(directory2)  
else:
    os.makedirs(directory2)   
    
fileList = glob.glob("./Images/*.*")

for id,fp in enumerate(fileList):
    filename, file_extension = os.path.splitext(fp)
    uid = filename.split('/')[-1]
    
    for prof in my_original_list:
        if prof['User_ID'] == uid:
            
            prof_age = prof['age']
            
            if (1 <= prof_age <= 18):
                new_file_extension = 'Child'
            elif (19 <= prof_age <= 45):
                new_file_extension = 'Adult'
            else:
                new_file_extension = 'Senior'
            
            copyfile(filename + ".jpg", './Label_Images_Age/'+ uid + '.' + str(id) + "." + new_file_extension +'.jpg')
            #copyfile(filename + ".jpg", new_file_extension +'/' + filename + ".jpg")

In [10]:
# For face detection we will use the Haar Cascade provided by OpenCV.
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

# For face recognition we will the the LBPH Face Recognizer 
recognizer = cv2.createLBPHFaceRecognizer()

In [11]:
def get_images_and_labels(path):
    # Append all the absolute image paths in a list image_paths
    
    image_paths = [os.path.join(path, f) for f in os.listdir(path)]
    # images will contains face images
    images = []
    # labels will contains the label that is assigned to the image
    labels = []
    #gender will contains 1 or 0 indecating male or female
    age =[]
    
    for image_path in image_paths:
        # Read the image and convert to grayscale
        try:
            image_pil = Image.open(image_path).convert('L')
            # Convert the image format into numpy array
            image = np.array(image_pil, 'uint8')
            # Get the label of the image
        except:
            pass
        
        nbr = int(os.path.split(image_path)[1].split(".")[1])
        age_current = os.path.split(image_path)[1].split(".")[2]
        print nbr
        
        # Detect the face in the image
        faces = faceCascade.detectMultiScale(image)
        # If face is detected, append the face to images and the label to labels
        try:
            for (x, y, w, h) in faces:

                ref_image = image[y: y + h, x: x + w]
                resized = cv2.resize(ref_image, (100, 100), interpolation = cv2.INTER_AREA)
                images.append(resized)
                labels.append(nbr)

                if age_current == 'Child':
                    age.append(0)
                elif age_current == 'Adult':
                    age.append(1)
                else:
                    age.append(2)
                    
                cv2.imshow("Adding faces to traning set...", resized)
                cv2.waitKey(50)
        except:
            pass
    # return the images list and labels list
    print "lables"
    print labels
    print "gender_current"
    print age
    
    return images, labels, age

In [12]:
images, labels, ageList = get_images_and_labels('Label_Images_Age')

cv2.destroyAllWindows()

331
1676
1168
164
589
1301
350
348
91
803
1504
165
465
1408
729
721
229
1110
383
889
33
794
439
564
565
342
448
300
1172
1698
293
208
816
1181
791
1130
1140
843
143
823
652
27
207
440
1149
1471
669
1175
1556
557
426
740
1688
946
1139
679
1475
1328
171
481
1654
790
1496
1137
1484
804
1452
1462
1042
1546
1470
363
414
619
63
357
1335
283
726
359
1501
614
1461
247
403
1536
1483
1129
854
1523
1352
494
773
325
1012
1428
1539
516
1436
306
31
1064
1057
498
849
340
241
1623
191
888
1564
520
1433
32
1100
259
1383
303
1368
6
1091
1248
1624
1672
1156
1403
1000
251
1027
71
418
530
750
1174
497
1531
416
734
1270
1240
1655
1619
979
22
622
44
272
525
1151
238
1407
553
1189
668
630
1594
1542
382
1599
1350
977
599
483
85
964
270
1597
873
146
457
1661
904
206
82
1697
728
1413
152
1438
1605
117
274
1554
1244
1506
192
1286
1650
628
1066
957
441
330
1261
242
289
24
1364
157
1544
329
1039
688
1311
1547
1430
945
1648
1399
1004
93
881
1178
343
1154
1034
1497
805
1264
11
1427
1246
2
1194
777
1282
709
587
354
25

In [13]:
images[0].shape

(100, 100)

## Making the Dataset 

In [17]:
res_images = []
res_age = []

for img in images:
    res_images.append(img.reshape(1,10000))    # (100*100 image size)

for age in ageList:
    res_age.append(np.array(age).reshape(1,))
    
res_images = np.concatenate(res_images, axis=0)
res_age = np.concatenate(res_age,axis=0)

In [18]:
res_images.shape

(1477, 10000)

In [20]:
res_age.shape

(1477,)

In [21]:
X,y = res_images,res_age

In [22]:
X_Train, X_Test = X[:1100], X[1100:]
y_Train, y_Test = y[:1100], y[1100:]


print y_Train.shape,y_Test.shape

(1100,) (377,)


In [23]:
# Logistic Regression
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
# load the iris datasets
#dataset = datasets.load_iris()
# fit a logistic regression model to the data
model = LogisticRegression(C=1e5)
model.fit(X_Train,y_Train)
print(model)
# make predictions
expected = y_Test
predicted = model.predict(X_Test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         2
          1       0.90      0.89      0.89       332
          2       0.26      0.30      0.28        43

avg / total       0.82      0.81      0.82       377

[[  0   2   0]
 [  1 294  37]
 [  0  30  13]]
